# LLM 함수 호출

In [20]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GEMINI_MODEL = os.getenv('GEMINI_MODEL')

In [21]:
# 데이터 변수 만들기
prod_database = { 
    "갤럭시 S24": {"재고": 10, "가격": 1_700_000}, 
    "갤럭시 S23": {"재고": 5, "가격": 1_300_000}, 
    "갤럭시 S22": {"재고": 3, "가격": 1_100_000}, 
} 

In [22]:
# is_product_available 함수
def is_product_available(product_name: str) -> bool:
  """특정 제품의 제고가 있는지 확인한다.
  
  Args:
      product_name: 제품명
  """
  if product_name in prod_database:
    if prod_database[product_name]["재고"] > 0:
      return True
  return False

In [23]:
# get_product_price 함수
def get_product_price(product_name: str)-> int: 
    """제품의 가격을 가져온다.

    Args:
        product_name: 제품명
    """
    if product_name in prod_database: 
        return prod_database[product_name]["가격"] 
    return None 

In [24]:
# place_order 함수
def place_order(product_name: str, address: str)-> str: 
    """제품 주문결과를 반환한다.
    Args:
        product_name: 제품명
        address: 배송지
    """
    if is_product_available(product_name): 
        prod_database[product_name]["재고"] -= 1 
        return "주문 완료" 
    else: 
        return "재고 부족으로 주문 불가" 

In [25]:
# 모델이 JSON 문자열로 알려준 함수와 실제 함수를 연결하기 위한 매핑 정보
function_repoistory = {
    "is_product_available": is_product_available,
    "get_product_price": get_product_price,
    "place_order": place_order
}
function_repoistory.values()

dict_values([<function is_product_available at 0x0000028B87CE5800>, <function get_product_price at 0x0000028B87DE6020>, <function place_order at 0x0000028B87DE6480>])

In [26]:
# 모델을 생성할 때 tools 필드에 함수 리스트 세팅
model = genai.GenerativeModel(
  model_name=GEMINI_MODEL,
  tools=function_repoistory.values() # 함수의 주소값이 저장되어 있음
)

In [27]:
# 메시지 주고받기
chat_session = model.start_chat()
prompt = "갤럭시 S24 판매 중인가요?"

response = chat_session.send_message(prompt)
# correct_response(response)
print(response.candidates[0].content)

parts {
  function_call {
    name: "is_product_available"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "\352\260\244\353\237\255\354\213\234 S24"
        }
      }
    }
  }
}
role: "model"



In [28]:
# 응답으로 function_call 데이터 타입이 반환되었으므로
# is_product_available(product_name="갤럭시 S24")와 같이 특정 함수가 호출되도록 해야 함
part = response.candidates[0].content.parts[0]

if part.function_call:
    function_call =  part.function_call
    function_name = function_call.name
    function_args = {k: v for k, v in function_call.args.items()}
    print(f"{function_name} args=>: {function_args}")

    # repository에서 key(이름)로 value(함수)를 가져온 후 딕셔너리 언패킹/패킹을 활용하여 함수를 호출
    # 함수를 호출한 결괏값은 function_response 형태의 Part 데이터로 전달하면서 모델이 메시지를 생성할 수 있음
    function_result = function_repoistory[function_name](**function_args)
    print(f"{function_name} result=>: {function_result}")

is_product_available args=>: {'product_name': '갤럭시 S24'}
is_product_available result=>: True


In [29]:
part = glm.Part(
    function_response=glm.FunctionResponse(
        name="is_product_available", 
        response={ 
            "content": function_result, 
        }, 
    )
)

print(part)
response = chat_session.send_message(part)
print("-----")
print(response.candidates[0].content)

function_response {
  name: "is_product_available"
  response {
    fields {
      key: "content"
      value {
        bool_value: true
      }
    }
  }
}

-----
parts {
  text: "\353\204\244, \355\214\220\353\247\244\355\225\230\352\263\240 \354\236\210\354\212\265\353\213\210\353\213\244. \n"
}
role: "model"

